In [4]:
import torch
import torch.utils.data as Data
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import pandas as pd

#Input data -> data reshape(converting to tensor) -> data loader -> 
#build CNN -> define optimizer & loss -> test data

#reading the training and testing data
train = pd.read('X.pt')
train_label = pd.read('y.pt')
test = pd.read('file_ids.pkl')

train_x = train.drop(['label'],axis=1,inplace=False)
train_y = train['label']

plt.imshow(train_x.ix[1,:].reshape(28,-1),cmap='gray')

train x = train x / 255.0
test = test / 255.0

#transform into tensor
train_tensor = torch.from_numpy(train_x.values.reshape(-1,28, 28)).float
test_tensor = torch.from_numpy(test.values.reshape(-1, 28, 28)).float

#data loader
torch_train = Data.TensorDataSet(data_tensor=train_tensor,
                                 target_tensor = torch.from_numpy(train_y.values))

loader = Data.DataLoader(dataset=torch_train, 
                        batch_size=5,
                        shuffle= True,
                        num_workers=2)

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.conv1 = nn.Sequential( 
            nn.Conv2d(in_channels=1,
                      out_channels=16,
                      kernel_size=3,
                      stride=1, #distance of stepping
                      padding=1) #(kernel - stride) /2
            nn.ReLU(),
            nn.Maxpool2d(kernel_size=2) #in how many steps we need to conclude
        )
        self.fc1 = nn.Linear(32*7*7, 120) #put it into a fully trained model
        self.fc2 = nn.Linear(120, 10)
        
    def forward(self,x):
        x= self.conv1(x)
        x= self.conv2(x)
        #stop conv
        x = x.view(x.size(0),-1)#size0 means batch, -1 means let PC calculate the pixel
        x=F.sigmoid(self.fc1(x))
        output = self.fc2(x)
        
        return output
        

AttributeError: module 'pandas' has no attribute 'read'

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data

#data route
data_direction = '/Users/zhousq/Documents/Semesters/2020_Fall/Computer_Vision/hw1/MNIST'

#data download
mnist = input_data.read_data_sets(data_direction, one_shot=True)

In [1]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
# options
dataset = 'mnist' # options: 'mnist' | 'cifar10'
batch_size = 64   # input batch size for training
epochs = 10       # number of epochs to train
lr = 0.01        # learning rate

In [7]:
# Data Loading
# Warning: this cell might take some time when you run it for the first time, 
#          because it will download the datasets from the internet
if dataset == 'mnist':
    data_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    trainset = datasets.MNIST(root='.', train=True, download=True, transform=data_transform)
    testset = datasets.MNIST(root='.', train=False, download=True, transform=data_transform)

elif dataset == 'cifar10':
    data_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    trainset = datasets.CIFAR10(root='.', train=True, download=True, transform=data_transform)
    testset = datasets.CIFAR10(root='.', train=False, download=True, transform=data_transform)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader  = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=0)

In [8]:
## network and optimizer
if dataset == 'mnist':
    num_inputs = 784
elif dataset == 'cifar10':
    num_inputs = 3072

num_outputs = 10 # same for both CIFAR10 and MNIST, both have 10 classes as outputs

class Net(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(Net, self).__init__()
        self.linear = nn.Linear(num_inputs, num_outputs)

    def forward(self, input):
        input = input.view(-1, num_inputs) # reshape input to batch x num_inputs
        output = self.linear(input)
        return output

network = Net(num_inputs, num_outputs)
optimizer = optim.SGD(network.parameters(), lr=lr)

In [9]:
def train(epoch):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = network(data)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test():
    network.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        #data, target = Variable(data, volatile=True), Variable(target)
        output = network(data)
        test_loss += F.cross_entropy(output, target, reduction='sum').item() # sum up batch loss
        #test_loss += F.cross_entropy(output, target, sum=True).item() # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [10]:
train(1)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.640927
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.447774
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.357716
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.496971
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.338677
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.289857
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.443109
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.356889
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.162691
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.355089
